# ----------------------------------------------------------------------

PROJECT'S TITLE

DATA SCIENTIST PROJECT RELATED TO THE ANALYSIS OF HOSPITALIZATION DATA 
 IN FRANCE DUE TO COVID-19.


PROJET/PURPOSE:


The UDACITY DATA SCIENTIST CAPSTONE project consists in building a data
 science project of my choice with two deliverables:
 
 - A Github repository of my work;

 - A blog post written for technical audience (my choice).


Considering the rebound of COVID-19 cases in France these last weeks,
 I decided to analyze data of COVID-19 progress in France with figures
 directly reported by the hospitals via an institutional site.

1 - I will define the problem I want to solve.
  
2 - I will analyze this problem through visualization and data exploration
  to have a better understanding of the process to implement.

3 - I will implement the algorithms and metrics to solve this problem.

4 - I will collect my results and conclude to what extend I solve the
    problem.

5 - I will propose a blog post to document the steps or my work.


The notebook here-below gathers my work for this purpose.

# ----------------------------------------------------------------------

SECTION 1 - DEFINITION OF THE PROBLEM TO SOLVE


Using data from institutional site, I will focus my attention on the
 hospitalization, i.e. how COVID-19 fills our hospitals with new
 patients and the capacity of our hospitals to absorb these new
 patients.

I will monitor the hospitalization in the context of the population and
 the rate of people positive to the COVID-19 and the mortality. I will
 work at the scale of the department and even the region. My attention
 will be particularly focused on my department and my region.

The problem to solve consists in providing threshold on parameters to 
 identify that would indicate when increase of hospitalization would
 significantly increase the risk of going in resuscitation or intensive
 care, or even the risk of death.

In that perspective, I will also try to propose a model for predicting
 the number of hospitalization. ............................................................................. TBC

For measuring efficiency of my solution, I propose following metrics:
............................................................................................................. TBD

In [ ]:
# ----------------------------------------------------------------------
# import libraries
import sys
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
# import pygal
# from pygal_maps_fr import maps

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# requires installing
# https://gtk-win.sourceforge.io/home/index.php/Main/Downloads

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - Data Exploration


I've gathered several data files reporting various information related
 to the epidemic of COVID-19 in France from the institutional web site:
 https://www.data.gouv.fr/fr/datasets

The source web site indicates that data have been gathered along the
 time from various formats and contents.

All the files used are under the License "Open Data Commons Open
 Database License (ODbL)". 

I downloaded the following data files the 29 December 2022. All data are
 stored in Github of this project.


FILE # 1

- Initial name: "donnees-vaccination-par-tranche-dage-type-de-vaccin-et-departement.csv"

- New name: "COVID-19_France_data_vaccin_population.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/donnees-vaccination-par-tranche-dage-type-de-vaccin-et-departement-region/"

- Last update: 13-Dec-2022

- License ID: 60bdce49abcc8f5dcb2fcb3b

- Brief description: Vaccination data by age bracket, type of vaccine
                     and department / region.

- Size: 146 Mb


FILE #2

- Initial name: "donnees-vaccination-par-pathologie.csv"

- New name: "COVID-19_France_data_vaccin_pathology.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/donnees-vaccination-par-pathologie-et-departement-region/"

- Last update: 13-Dec-2022

- License ID: 60bdce48e306a187d32ee2c1

- Brief description: Vaccination data per pathology and department
                     / region.

- Size: 1.2 Mb


FILE #3

- Initial name: "table-indicateurs-open-data-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_epidemic_indicators.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/synthese-des-indicateurs-de-suivi-de-lepidemie-covid-19/"

- Last update: 28-Dec-2022

- License ID: 60190d00a7273a8100dd4d38

- Brief description: COVID-19 epidemic monitoring indicators summary

- Size: 11.2 Mb


FILE #4

- Initial name: "vacsi-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_vaccin_indicators.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/r/735b0df8-51b4-4dd2-8a2d-8e46d77d60d8"

- Last update: 28-Dec-2022

- License ID: 6010206e7aa742eb447930f7

- Brief description: Data related to people vaccinated against COVID-19
                   per department.

- Size: 6.2 Mb


FILE #5

- Initial name: "vacsi-tot-dep-2022-12-28-19h00.csv"

- New name: "COVID-19_France_data_vaccin_indicators_tot.csv"

- Steady URL: "https://www.data.gouv.fr/fr/datasets/r/7969c06d-848e-40cf-9c3c-21b5bd5a874b"

- Last update: 28-Dec-2022

- License ID: 6010206e7aa742eb447930f7

- Brief description: Data related to cumulative people vaccinated against
                   COVID-19 per department.

- Size: 10 Kb


COMMENT:

Data file were renamed to avoid being dependent of the date that may be
 notified in the file's name and to simplify and gather the designation
 of these files in the workspace.


All these files are known to be of type 'txt/csv'. They are fulfilled 
 in french such that I was required to open these files under Unicode
 UTF-8 format.


Lets explore these files through the notebook!

# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS

# Purpose: Define the main directory of the data files: 'work_dir'
# Instruction: Modify value of 'work_dir' according to your workspace.

dir_1 = r"C:\Users\to202835\OneDrive - ATR\_exploitation\formation"
dir_2 = r"\2022\Udacity_DataScience\06_DataScientist Capstone\v1.3.0"
work_dir = dir_1 + dir_2

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS

# Purpose: Build a function that read input csv files
# Discussion: Though some of the used data files are also available in
#             json format, I prefered get all my data files in the same
#             format to ease the read.

def read_csv(filename,
             dtype_values={},
             filedir=work_dir,
             separator=',',
             encod_errors='ignore'):
    '''
    DESCRIPTION
        Read input csv files
    INPUT
        filename (string) is the name of the data file to read;
        dtype_values (dictionary) is the set of information to enforce
                     read of selected columns in a given format to 
                     possibly avoid warning message of Low memory:
                     default = none;
        filedir=work_dir (string) is the path of the data files 
                         directory.
                         default = work_dir (previously defined)
    OUTPUT
        df (pandas dataframe) is the dataframe resulting from the read
           of the input data file. If the read fails, it returns an 
           empty dataframe.
    '''

    # Build the complete file path
    filepath = "\\".join((filedir, filename))

    # trying read the csv data file
    try:
        df = pd.read_csv(filepath.replace('\\','/'),
             dtype=dtype_values, sep=separator,
             encoding_errors=encod_errors,
             low_memory=False)
        print("- Opening '{}'".format(filename))
    except:
        df = pd.DataFrame([])
        print("- Unable to open '{}'".format(filename))
        raise

    return df

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS - FILE #1

# define input features
f_file_1 = r"COVID-19_France_data_vaccin_population.csv"
'''Discussion
Initial read of the file report an issue on format of column #2 that 
 causes a low memory warning. The issue was solved by setting the
 appropriate format of some columns with mixed types (identified after
 the initial opening and analysis of the file) and by using separator
 ";".
'''
dtypes = {'date_reference': str,
 'semaine_injection': str,
 'region_residence': str,
 'libelle_region': str,
 'departement_residence': str,
 'libelle_departement': str,
 'classe_age': str,
 'libelle_classe_age': str,
 'type_vaccin': str,
 'date': str}  # These columns have a mixed type of data

# Read the file
df_file_1 = read_csv(f_file_1, separator=";")
# Get a view on this file
df_file_1.head(3)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS - FILE #2

# define input features
f_file_2 = r"COVID-19_France_data_vaccin_pathology.csv"
# Read the file
df_file_2 = read_csv(f_file_2, separator=";")
# Get a view on this file
df_file_2.head(3)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS - FILE #3

# define input features
f_file_3 = r"COVID-19_France_data_epidemic_indicators.csv"
'''Discussion
Initial read of the file report an issue on format of column #2 that 
 causes a low memory warning. The issue was solved by setting the
 appropriate format of some columns with mixed types (identified after
 the initial opening and analysis of the file).
'''
dtypes = {"date": str,
          "dep": str,
          "lib_dep": str,
          "lib_reg": str}  # These columns have a mixed type of data

# Read the file
df_file_3 = read_csv(f_file_3, dtype_values=dtypes)
# Get a view on this file
df_file_3.head(3)


In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS - FILE #4

# define input features
f_file_4 = r"COVID-19_France_data_vaccin_indicators.csv"
dtypes = {"dep": str}  # column 'dep' has mixed types. So I enforce the
                       #  format while reading the csv file to avoid a
                       #  low memory warning message. 
# Read the file
df_file_4 = read_csv(f_file_4,
                         dtype_values=dtypes,
                         separator=";")
# Get a view on this file
df_file_4.head(3)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - DATA ACCESS - FILE #5

# define input features
f_file_5 = r"COVID-19_France_data_vaccin_indicators_tot.csv"
dtypes = {"dep": str}  # column 'dep' has mixed types. So I enforce the
                       #  format while reading the csv file to avoid a
                       #  low memory warning message. 
# Read the file
df_file_5 = read_csv(f_file_5,
                         dtype_values=dtypes,
                         separator=";")
# Get a view on this file
df_file_5.head(3)

# ----------------------------------------------------------------------

# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA


Before going deeper in the understanding and definition of every 
 parameters of these files, I'd like to have a view on the type of data
 and availability of valid data.
In addition, in the perspective of merging these files, I will try
 finding some links that would allow me benefit from a large quantity of 
 data for pursuing the analysis and solve my problem.

I'm starting by building a function that would allows me checking some 
 charactertics of the data set per column.
It will help me identifying necessary processing of the raw data and
 checking that I produce relative appropriate data (format and value).

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA

# Build a function to analysis the type and availability of valid data.

def display_columns_info(df):
    '''
    DESCRIPTION
        Display some information per column related to the data set
    INPUT
        df is the dataframe
    OUTPUT
        nil (only display)    
    '''
    
    # Measure length of the column's title only for presentation purpose
    max_len = 0
    cols = np.array(df.columns)
    for c in cols:
        if len(c) > max_len:
            max_len = len(c)
    size = max_len + 2  # I will create a regulat length for label field.

    # Display information per column        
    field_1 = "column label" + ' '*(size - 11)
    field_size = 10
    field_2 = "      size "
    field_3 = "      type "
    field_4 = "   nb uniq  rate uniq "
    field_5 = "    nb nan   rate nan "
    field_6 = "    nb inf   rate inf "
    field_7 = "     other "
    # Introducing the results
    m = field_1 + field_2 + field_3 + field_4 + field_5 + field_6 + field_7
    print(m)

    # Then run along the columns to perform and give the analysis per column.
    for col in cols:
        # column's size
        col_size = df[col].shape[0]
        # get column's type        
        col_type = df[col].dtypes
        cot_typ = str(col_type)
        # get number and rate of unique values
        values_uniq = np.array(pd.unique(df[col]))
        nb_uniq = len(values_uniq)
        rate_uniq = 100 * nb_uniq / col_size
        # get number and rate of nan values
        nb_nan = df[col].isna().sum()
        rate_nan = 100 * nb_nan / col_size
        # get number and rate of infinite values
        nb_inf = 0
        mixed_types = []
        # Run along values of the column
        for value in np.array(df[col].values):
            # Count infinite value
            if (value == np.inf) or (value == -np.inf):
                nb_inf += 1
            # Check consistency of the value type with its column
            if " " in str(type(value)):
                val_type = str(type(value)).split("'")[1]
            else:
                if "." in str(type(value)):
                    val_type = str(type(value)).split(".")[1][:-1]
                else:
                    val_type = str(type(value))
            if (val_type != cot_typ):
                if val_type not in mixed_types:
                    mixed_types.append(val_type)

        # Compute the rate of infinite values in the column
        rate_inf = 100 * nb_inf / col_size

        # Report mixed types if any
        msg_mixed_types = ''
        if len(mixed_types) > 1:
            msg_mixed_types = '\tMixed types: ' + str(mixed_types)

        # Check among 'date' values that they are all strings with a length
        #  of 10.
        mem_unexp_dates = []
        msg_unexp_dates = ''
        if col == 'date':
            for value in values_uniq:
                if (not isinstance(value, str)) or (len(value) != 10):
                    mem_unexp_dates.append(value)
            if len(mem_unexp_dates) > 0:
                msg_unexp_dates = '\tWrong date format: ' + str(mem_unexp_dates)

        # Display result
        dec = 2
        print('{}{} {}{} {}{} {}{}{}{} % {}{}{}{} % {}{}{}{} % {} {}'.format(
            col, ' '*(size - len(col)),
            ' ' * (field_size - len(str(col_size))), col_size,
            ' ' * (field_size - len(str(col_type))), col_type,
            ' ' * (field_size - len(str(nb_uniq))), nb_uniq,
            ' ' * (field_size -1 - len(str(round(rate_uniq, dec)))), round(rate_uniq, dec),
            ' ' * (field_size - len(str(nb_nan))), nb_nan,
            ' ' * (field_size -1 - len(str(round(rate_nan, dec)))), round(rate_nan, dec),
            ' ' * (field_size - len(str(nb_inf))), nb_inf,
            ' ' * (field_size -1 - len(str(round(rate_inf, dec)))), round(rate_inf, dec),
            msg_unexp_dates,
            msg_mixed_types)
            )

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #1

# Check the data
display_columns_info(df_file_1)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #1


Result of the check of the dataframe 'df_file_1' reveals that:

- 'date_reference', 'semaine_injection', 'region_residence', 
  'libelle_region', 'departement_residence', 'libelle_departement',
  'classe_age', 'libelle_classe_age', 'type_vaccin' and 'date' are
  defined as 'object'.

  ACTION: Think to convert these columns in dummies if needed for 
   modeling or other use.

- 'population_insee' contains 1.64% nan values.

   ACTION: Monitor the need to keep these rows or replace nan!

- 'effectif_cumu_1_inj', 'effectif_cumu_termine', 'taux_cumu_1_inj' and
  'taux_cumu_termine' contain around 50% nan values.

   ACTION: Monitor the need to keep these rows or replace nan if
    possible!

- 'effectif_1_inj', 'effectif_termine', 'taux_1_inj', 'taux_termine',
  'effectif_rappel', 'effectif_cumu_rappel', 
  'effectif_rappel_parmi_eligible', 'effectif_eligible_au_rappel',
  'taux_rappel', 'taux_cumu_rappel' and 'taux_cumu_rappeleli'
  contain more than 70% nan values; I would recommend removing 
  this column because almost the whole column has no consistent 
  values.
  
  ACTION: Remove this column immediately


Possible improvement:
Automatize cleaning according the the level of missing values.

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #1 - CLEANING

# Remove columns almost empty (at least 50% of nan):
list1 = ['effectif_cumu_1_inj', 'effectif_cumu_termine',
         'taux_cumu_1_inj', 'taux_cumu_termine', 'effectif_1_inj',
         'effectif_termine', 'taux_1_inj', 'taux_termine', 
         'effectif_rappel', 'effectif_cumu_rappel', 
         'effectif_rappel_parmi_eligible', 'effectif_eligible_au_rappel',
          'taux_rappel', 'taux_cumu_rappel', 'taux_cumu_rappeleli']
df_file_1.drop(columns=list1, inplace=True)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #2

# Check the data
display_columns_info(df_file_2)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #2


Result of the check of the dataframe 'df_file_2' reveals that:

- 'date', 'regroupement_pathologie', 'pathologie', 'region_residence',
  'libelle_region', 'departement_residence' and 'libelle_departement'
   are defined as 'object'.
   
   ACTION: Think to convert these columns in dummies if needed for 
    modeling or other use.

- 'libelle_region' contains 0.85% nan values.

- 'population_patho_cartographie' contains 2.22% nan values.

- 'effectif_1_inj_pathologie', 'effectif_termine_pathologie', 
  'taux_1_inj_pathologie', 'taux_termine_pathologie' and
  'effectif_eligible_rappel_patho' contain 2.42% nan values.

- 'effectif_rappel_parmi_eli_patho', 'taux_rappel_pathologie' and
  'taux_rappel_eligible_pathologie' contain 3.15%nan values.

- 'ordre' contains 12.28% nan values.

ACTION: Monitor the need to keep these rows with nan or replace nan!


=> No immediate action required! 

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #3

# Check the data
display_columns_info(df_file_3)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #3


Result of the check of the dataframe 'df_file_2' reveals that:

- Although enforcing format of 'dep', 'date', 'lib_dep' and 'lib_reg'
   into str while reading the csv file, type of these columns is still
   'object'.

   ACTION: Think to convert these columns in dummies if needed for 
    modeling or other use.

- 'incid_hosp', 'incid_rea', 'incid_dchosp' and 'reg_incid_rea' contain
   0.1% nan values.

   ACTION: Monitor the need to keep these rows or replace nan!

- 'tx_pos', 'tx_incid', 'pos' and 'pos_7j' contain 5.81% of nan values.

   ACTION: Monitor the need to keep these rows or replace nan!

- 'R' contains 84.94% nan values; I would recommend removing this
   column because too many values are missing without possibility to 
   replace missing values with a relevant value.

   ACTION: remove this column immediately.

- 'cv_dose1' contains 99.9% nan values; I would recommend removing this
   column because almost the whole column has no consistent values.
   
   ACTION: remove this column immediately.

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #3 - CLEANING

# Remove columns with a lot of missing data: 'R' and 'cv_dose1'.
list3 = ['R', 'cv_dose1']
df_file_3.drop(columns=list3, inplace=True)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #4

# Check the data
display_columns_info(df_file_4)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #4


Result of the check of the dataframe 'df_file_4' reveals that:

- 'dep' and 'jour' are defined as 'object' while I've enforced 'dep' 
  into string format.

  ACTION: Think to convert these columns in dummies if needed for 
    modeling or other use.

- There is neither nan nor infinite values in this dataframe.

  ACTION: nil

- To be homogeneous with other dataframes, 'jour' must be replaced by
  'date'.
  
  ACTION: replace column's label 'jour' by 'date' (same format)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #4 - CLEANING

# Rename column 'jour' as 'date'
df_file_4.rename({'jour': 'date'}, axis=1, inplace=True)

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #5

# Check the data
display_columns_info(df_file_5)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #5


Result of the check of the dataframe 'df_file_5' reveals same comments
that for 'df_file_4'.

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - TYPE AND VALIDITY OF DATA - FILE #5 - CLEANING

# Rename column 'jour' as 'date'
df_file_5.rename({'jour': 'date'}, axis=1, inplace=True)

# ----------------------------------------------------------------------

SECTION 2 - ANALYSIS - EXPLORATION AND VISUALIZATION


AT this level, I'd like to merge all dataframes to benefit the maximum
 data to work on later. But is it possible?
My main concern, since I use data file with time series, is that they 
 all covers the common time period long enought to keep a lot of data 
 to work on.

Let's have a view on the time series on every input file.

Once we would have identified the input data files that I can keep or
 no, I will merge them by the columns 'date' and 'dep'.

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - ANALYSIS - EXPLORATION - FILES MERGING

# Basically, I want to know how many unique dates owns every dataframe:
print('Number of unique values of date per input data file:')
print("File #1:", len(np.unique(np.array(df_file_1['date']))))
print("File #2:", len(np.unique(np.array(df_file_2['date']))))
print("File #3:", len(np.unique(np.array(df_file_3['date']))))
print("File #4:", len(np.unique(np.array(df_file_4['date']))))
print("File #5:", len(np.unique(np.array(df_file_5['date']))))
# note: Applying the function 'display_columns_info' give me already
#  the same result but it allows to summarize it here.

# Result: I see that merging all files would give me only one date.
#         So I need to identify clearly the covering time period of
#         every files to see which ones I could merge.
#         A priori, I would keep only files 3 and 4, let's check that.
#         Files 2 and 5 return only one date: ['2022-12-04']

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - DATA VISUALIZATION - PLOTTING TIME SERIES

# Build a generic function for displaying time series

def plot_time_series(df, label: str, departments: list):
    '''
    DESCRIPTION
        Plot a line chart of figures for the given label and the 
         selected list of departments.
    INPUT
        df is the pandas dataframe;
        label (string) is the name of the column for which I want a
              plot;
        departments (list) is the list departments for which I want
                    a plot.
    OUTPUT
        nil (only display)
    '''

    # If not already done, I group df by department and timestamp
    # df.groupby(['dep', 'timestamp'])  # If no dummy applied
    df.groupby(['dep', 'date'])  # If no dummy applied

    # get the uniq list of the selected departments, just in case of...
    departments = np.unique(departments)  # just in case of 
    
    df_plot_list = []  # list to store data for plotting of every
                       #  selected departments.
    # Run along the list of departments
    for dep in departments:
        # Get the values of the label along the time
        df_dep = pd.DataFrame([])
        df_dep = df[df['dep'] == dep]

        # Get the time for this dep:
        x_val = np.array(df_dep['date'])
        
        # get the values of the label for this dep
        y_val = np.array(df_dep[label])

        # Create a dataframe for the plot
        df_plot = pd.DataFrame(dict(x = x_val, y = y_val))

        # store the data for plotting
        df_plot_list.append(df_plot)

    # Plotting
    # declare the figure
    fig = go.Figure()

    # Run along the list of dataframes containing plotting data
    for i, df_plot in enumerate(df_plot_list):
        # Add a line of result per dataframe
        fig.add_trace(go.Scatter(x=np.array(df_plot['x']),
                                y=np.array(df_plot['y']),
                                mode='lines+markers',
                                name='Dept. ' + str(departments[i])
                                )
                    )
    # Define the context of the plot
    titre = df_dict_labels[label] + "\n(per department)"
    fig.update_layout(title=titre,
                    xaxis_title="Time",
                    yaxis_title=df_dict_labels[label],
                    legend=dict(traceorder='reversed'))
    fig.show()

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - DATA VISUALIZATION - PLOTTING TIME SERIES

# I build a function to format x/y values for plotting lines

def get_df_dates(dfs, label='timestamp'):
    '''
    DESCRIPTION
        return dataframes with min/max of the column label in abscisse
         and index in ordonate. For visualization purpose
    INPUT
        dfs (list) is a list of dataframes where I will search for values 
            in the column named label;
        label (string) is the name of the column where I will lokk for 
              timestamp values; default = 'timestamp'.
    OUTPUT
        dfs_return (list) is a list of dataframes such that every
                   dataframe is defined like:
                   {'x' = [values of ‘dates’ from the dataframe] }
                   {'y' = [position of the dataframe in the list] }
    '''
    dfs_return = []

    # Run along the list of dataframes
    for i, df in enumerate(dfs):
        '''
        # sort timestamp by ascending order
        df.sort_values(by='timestamp', ascending=True, inplace=True)
        # get min and max timestamp values given by the dataframe
        min_ts, max_ts = df['timestamp'].min(), df['timestamp'].max()
        # get date (str) equivalent for these min/max timestamps
        min_date = get_df_related_value(df, 'timestamp', 'date', min_ts)
        max_date = get_df_related_value(df, 'timestamp', 'date', max_ts)
        # build a dataframe with min/max dates and index related to 
        #  the dataframe.
        df_ts = pd.DataFrame(dict(
                x = [min_date, max_date],
                y = [i+1, i+1]
        ))
        '''
        # get X/Y values
        x_values = df['date'].tolist()
        y_values = [i+1] * len(x_values)
        # format X/Y for ploting
        df_ts = pd.DataFrame(dict(
                x = x_values,
                y = y_values
        ))
        
        # Store the dataframe into the the list of dataframes
        dfs_return.append(df_ts)

    return dfs_return

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - DATA VISUALIZATION - PLOTTING TIME SERIES

# I build a function to format x/y values for plotting lines

def plot_df_time_periods(dfs):
    '''
    DESCRIPTION
        Plot a line chart showing the time period of every input 
         dataframe.
    INPUT
        dfs (list) is a list of dataframes such that every
            dataframe is defined like:
            {'x' = [values of ‘dates’ from the dataframe] ] }
            {'y' = [position of the dataframe in the list] }
    OUTPUT
        nil (only display)
    '''
    # declare the figure
    fig = go.Figure()

    # Run along the list of dataframes containing plotting data
    for i, df in enumerate(dfs):
        # Add a line of result per dataframe
        fig.add_trace(go.Scatter(x=np.array(df['x']),
                                 y=np.array(df['y']),
                                 mode='lines+markers',
                                 name='file' + str(i+1)
                                )
                    )
    # Define the context of the plot
    titre = 'Difference of time period covered per data file'
    fig.update_layout(title=titre,
                    xaxis_title="time line",
                    yaxis_title="File#",
                    legend=dict(traceorder='reversed'))
    fig.update_yaxes(range=[0.5, len(dfs)+0.5])
    fig.show()

In [ ]:
# ----------------------------------------------------------------------
# SECTION 2 - DATA VISUALIZATION - PLOTTING TIME SERIES

# plot time period for every input dataframe

# Define the list of input dataframes
df_list = [df_file_1, df_file_2, df_file_3, df_file_4, df_file_5]

# Get data for plotting
dfs = get_df_dates(df_list)

# Plot the time periods
plot_df_time_periods(dfs)

In [ ]:
# ----------------------------------------------------------------------
# DATA PRE-PROCESSING - Merging files


# Build a function that provide the list of common values through
#  several dataframes for a given column's label.

'''Discussion
It may take a long time so there is certainly means to re-factorize 
 this part of the code.
'''

def get_common_values(dfs, label):
    '''
    DESCRIPTION
        get common values on the selectec label through all given 
         dataframes
    INPUT
        dfs (list) is the list of all dataframes.
        label (string) is the name of the column for which I want
         find common values.
    OUTPUT
        common_values is a numpy array of the common values
    '''
    common_values, all_values = [], []

    # run along every dataframe to find all values through all
    #  dataframes.
    for df in dfs:
        # get an array of values of the selected label
        values = np.unique(np.array(df[label]))
        # run along these values 
        for value in values:
            # if the value is not in the list of all values, I add it 
            #  to this list
            if value not in all_values:
                all_values.append(value)
    # all_values contains the list of all values of the column label 
    #  contained in all dataframes.

    # Run along all values to find only common values through all
    #  dataframes.
    for value in all_values:
        cnt = 0  # count the number of occurrence of this value
        for df in dfs:
            values = np.unique(np.array(df[label]))
            if value in values:
                cnt += 1
        if cnt == len(dfs):
            common_values.append(value)
    
    return np.array(common_values)



# Test function get_common_values
df1 = pd.DataFrame({'date': ['2020-05-12', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-23']})
df2 = pd.DataFrame({'date': ['2020-05-13', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-23']})
df3 = pd.DataFrame({'date': ['2020-05-13', '2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22', '2022-02-24']})
toto = get_common_values([df1, df2, df3], 'date')
print('computed {}\nexpected {}'.format(toto, np.array(['2020-06-13', '2021-07-06', '2021-08-01', '2022-01-22'])))

In [ ]:
# ----------------------------------------------------------------------
# DATA PRE-PROCESSING - Merging files
'''
# Here, i'm checking the range of the dates, to ensure that I will have
# access to most recent data.
common_dates = get_common_values([df_file_3, df_file_4], 'date')
df_dates = pd.DataFrame({"date": common_dates})
df_dates.sort_values(by="date", ascending=True, inplace=True)

# It finally appears that it ends at 27-Dec-2022 so I can use this
#  combination.
print(df_dates)
'''

In [ ]:
# ----------------------------------------------------------------------
# DATA PRE-PROCESSING - Merging files

# Merge the dataframes with 'date' and 'dep' as common columns.
print(df_file_3.shape)
print(df_file_4.shape)
df = pd.merge(df_file_3, df_file_4, on=['date', 'dep'], how='inner')
print(df.shape)

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - Post merging

# get the list of columns of the resulting merged dataframe
df.columns

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - Post merging

# check the dataframe 
display_columns_info(df)

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - Post merging

# remove nan from the dataframe
print(df.shape)
df.dropna(axis='index', how='any', inplace=True)
print(df.shape)

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - Post merging

# check the dataframe 
display_columns_info(df)

# ----------------------------------------------------------------------

DATA UNDERSTANDING - Post merging

This file contains a combination of main indicators for monitoring of
 the COVID-19 epidemic in France since 18 March 2020. Data are provided
 per department and per region of France. The file reports information
 related to COVID-19 testing campaign and patients at hospital.

# ----------------------------------------------------------------------

DATA UNDERSTANDING - Data Description - df_file_3:


I present only remaining data at this stage.

- Description of data - context:

	'date'    = date (object) when the information is given:
			    YYYY-MM-DD;

	'dep'     = number (str or int)of the french department;	

	'reg'     = number (int) of the french region;

	'lib_dep' = name (object) of the french department;

	'lib_reg' = name (object) of the french region.


- Description of data - Hospital situation:

	'hosp'         = number (int) of patients currently hospitalized
					  due to COVID-19;

	'incid_hosp'   = number (float) of new hospitalized patients in
				     the last 24h;

	'rea' is the   = number (int) of patients currently in
					 resuscitation or intensive care unit;

	'incid_rea'    = number (float) of new patients who were admitted
				     to the resuscitation unit in the last 24h;

	'rad'          = cumulative number (int) of patients who where
					  hospitalized for COVID-19 but back to home due
					  to improvement of their health;

	'incid_rad'    = number (float) of the patients back to home in
					 the last 24h;

	'reg_rea'       = undefined (int);

	'reg_incid_rea' = undefined (float).


- Description of data - decease due to COVID-19:

	'dchosp'       = number (int) of decease at hospital;

	'incid_dchosp' = number (float) of new patients deceased at the
					 hospital in the last 24h.


- Description of data - tests:  
  
	'pos'      = number (float) of people declared positive (D-3 date
	             of test);

	'pos_7j'   = number (float) of people declared positive on a week
				  (D-3 data of test);


- Description of data - COVID-19 epidemic monitoring indicators:  
  
	'tx_pos'   = Positivity rate (float) is the number of people tested
			      positive (RT-PCR or antigenic assay) for the first
			      time in the last 60 days over the number of people
			      tested (positive or negative) on a given period,
			      without being tested positive in the last 60 days;
				  
	'tx_incid' = Incidence rate (float) is the number of people tested
			      positive (RT-PCR or antigenic assay) for the first
				  time in the last 60 days over the size of population;
				it is given for 100 000 of inhabitants;
				  
	'TO' 	   = Occupancy rate (float) is the number of hostpitalized
		         COVID-19 patients over the initial number of beds at
				 hospital (before increase of this number).


Comment:
'reg_rea' and 'reg_incid_rea' are not defined.
I recommend removing them because I don't know their meaning.

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - cleaning dataframe

# As recommended, I remove useless data
df.drop(columns=['reg_rea', 'reg_incid_rea'], inplace=True)

# ----------------------------------------------------------------------

DATA UNDERSTANDING - Data Description - df_file_4:


'date' = date (object) when the information is given: YYYY-MM-DD;
	
'dep' = number (str or int)of the french department;

'n_dose1' = number of people who received their first dose of vaccine

'n_complet' = number of people who received a primo-vaccination

'n_rappel' = number of people primo-vaccinated who received at least
             one booster dose prescribed for specific reason by a 
             health professional.

'n_2_rappel' = number of people primo-vaccinated who received two
               booster doses prescribed for specific reason by a health
               professional.

'n_3_rappel' = number of people primo-vaccinated who received three
               booster doses prescribed for specific reason by a health
               professional.

'n_rappel_biv' = number of people vaccinated with a booster dose
                 adjusted to omicron variants.

'n_cum_dose1' = cumulative number of people who received one dose of
                vaccin.

'n_cum_complet' = cumulative number of people who received one dose of
                  vaccin.
 
'n_cum_rappel' = cumulative number of people primo-vaccinated who 
                 received at least one booster dose prescribed for
                 specific reason by a health professional.

'n_cum_2_rappel' = cumulative number of people primo-vaccinated who
                   received at least one booster dose prescribed for
                   specific reason by a health professional.

'n_cum_3_rappel' = cumulative number of people primo-vaccinated who
                   received three booster doses prescribed for specific
                   reason by a health professional.

'n_cum_rappel_biv' = cumulative number of people vaccinated with a
                     booster dose adjusted to omicron variants.

'couv_dose1' = vaccination coverage with first dose
               = n_cum_dose1 / pop

'couv_complet' = vaccination coverage with primo-vaccination
                 = n_cum_complet / pop

'couv_rappel' = vaccination coverage with first booster dose 
                = n_cum_rappel / pop

'couv_2_rappel' = vaccination coverage with second booster dose 
                  = n_cum_2_rappel / pop

'couv_3_rappel' = vaccination coverage with third booster dose 
                  = n_cum_3_rappel / pop

'couv_rappel_biv' = vaccination coverage with a booster dose adjusted
                    to omicron variants.

In [ ]:
# ----------------------------------------------------------------------
# DATA UNDERSTANDING - Post merging

# Create a dictionary with a shot description of every label for further
#  use during visualization of results.
df_dict_labels = {
    'dep': 'Dept.',
    'date': 'Date',
    'reg': 'Region',
    'tx_pos': 'Positivity rate',
    'tx_incid': 'Incidence rate',
    'TO': 'Hospital occupancy rate',
    'hosp': 'Hospitalized People',
    'rea': 'People in resuscitation/Intensice care',
    'rad': 'Cumulative hospitalized people back to home',
    'dchosp': 'Death at hospital',
    'reg_rea': 'unknown',
    'incid_hosp': 'Nes hospitalization in last 24h',
    'incid_rea': 'New people in resuscitation/Intensice care in last 24h',
    'incid_rad': 'New people hospitalized back to home in last 24h',
    'incid_dchosp': 'New death in last 24h',
    'reg_incid_rea': 'unknown',
    'pos': 'Positive case (J-3)',
    'pos_7j': 'Positive case on a week',
    'n_dose1': 'Vaccinated with 1 dose',
    'n_complet': 'Vaccinated with 2 doses',
    'n_rappel': 'Vaccinated with 1 booster dose',
    'n_2_rappel': 'Vaccinated with 2 booster doses',
    'n_rappel_biv': 'Vaccinated with 1 booster dose (adj. Omicron)',
    'n_3_rappel': 'Vaccinated with 3 booster doses',
    'n_cum_dose1': 'Cumulative vaccinated with 1 dose',
    'n_cum_complet': 'Cumulative vaccinated with 2 doses',
    'n_cum_rappel': 'Cumulative vaccinated with 1 booster dose',
    'n_cum_2_rappel': 'Cumulative vaccinated with 2 Booster doses',
    'n_cum_rappel_biv': 'Cumulative vaccinated with 1 booster dose (adj. Omicron)',
    'n_cum_3_rappel': 'Cumulative vaccinated with 3 Booster doses',
    'couv_dose1': 'Vaccionation coverage with 1 dose',
    'couv_complet': 'Vaccionation coverage with 2 doses',
    'couv_rappel': 'Vaccionation coverage with 1 booster dose',
    'couv_2_rappel': 'Vaccionation coverage with 2 booster doses',
    'couv_rappel_biv':'Vaccionation coverage with 1 booster dose (adj. Omicron)',
    'couv_3_rappel': 'Vaccionation coverage with 3 booster doses'
}

# ----------------------------------------------------------------------

DATA CLEANING - Defining the needs

As far as possible, I'd prefer working on dataset with the minimum
 missing values.

So I will clean the data set before iniating processing of data.

As I would like to build a prediction model, I'd like to use the
 cleanest data set possible, without invalid data (nan or infinite),
  without new columns that would be only computation from existing data.
In addition, as a priori, the prediction model will have to predict
 figures from figures, I will have to remove columns of string (unless
 I need to use through dummy values).

Before adding new computed parameters to the dataframe, I copy the
 dataframe merged and cleaned for further use in modeling.


First, it consists in building a function to clean the data set from
 unexpected values (nan, infinite) from the new computed data set.

# ----------------------------------------------------------------------

DATA CLEANING - Detailing the actions

Clean the data will first consists in removing useless data such as:

Remove string values by their equivalent in numerical values when
 possible before adding dummy values (only if needed). As far as I can,
 I would prefer find a simple substitute rather than add simple but
 numerous data because it remains easier to handle for plotting and
 modeling.

- Replacing string format of 'dep' by a pure numerical value (int);
  two corner cases to consider: Corse with ids 2A and 2B; I propose
  to replace them by 201 and 202.
  ACTION: build a function that replace string values of dep by int
          values.

- 'lib_dep' and 'lib_reg': Indeed, I could use only the index number
  and use a dictionary if needed to display the label of a deparment
  of a region.
  ACTION: Build dictionaries that give the label of department and
          region according to their id number. Then build a function
          that allows replacing an id or a list of ids by the labels.

- Replace the date by timestamp values; I propose to compute this
  timestamp as the number of second passed from year 0 to the date
  (string); it would ease the use when considering analysis and
  display with time series.
  ACTION: Build a function that compute values of timestamp, adding
          values in the dataframe.

- Finally, I will study the opportunity to remove columns and rows
  with missing values.
  ACTION: Analyze content of the dataframe to detect invalid data
          and decide if a can keep them or no.
          After a first modeling, I could also check the weight of
          every parameters to adjust/reduce content of the dataframe
          for modeling purpose. It leads to think a dataframe that
          would be specific for modeling.

Once cleaned, I will copy the dataframe for modeling.

# ----------------------------------------------------------------------

DATA CLEANING - Building Department & Region dictionaries

id of french departments and region shall be convert into numerical
 values before building their dictionaries.

Then labels of the departments and regions can be removed from the
 dataframe after having created a dictionary to find the label from
 the id.

This action consist in following steps:

- Replace/ensure that ids of department and region are numerical 
  values ((int) with no string.

- Build two dictionaries (department and region) that give the label
  from the id.

- Build two functions (department and region) that allows questioning
  these dictionaries to get the label from the id.

- Build two functions (department and region) to generalize and
  allowing getting a list of labels from a list of ids.
  
- Finally, once I have these dictionaries and functions, I can
  remove the columns related to labels of departments and regions.

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# I create a function that allows replacing str by int values

def replace_str_id(df, label):
    '''
    DESCRIPTION

    INPUT
        df is the pandas dataframe
        label (string) is the label of the column to process
              'dep' or 'region'.
    OUTPUT
        df is the input pandas dataframe with string values of label
        replace by int values.

    comment:
        think about specific Corse departments 2A and 2B
        that I propose to replace by 201 and 202.

    '''
    dico = {'2A': 201, '2B': 202}

    # get unique values of the label
    values = np.array(df[label].values)

    convert = []
    # Run along the values for conversion from str to int
    for value in values:
        # It try to refers to the dictionary for Corse's deps
        try:
            # basically, I try to convert str to int
            c = int(value)
        except:
            # when it fails,
            if value in dico:
                c = dico[value]
            else:
                # otherwise I apply value zero
                c = 0
        '''
        if (value in dico_keys):
            c = dico[value]
        else:
        '''
        convert.append(c)

    df[label] = np.array(convert)
    df[label] = df[label].astype(int)  # Ensure format for the columns.
                                       # Do not put before otherwise,
                                       #  it fails to convert 2A and 
                                       #  2B and display and warning

    return df

# check the function
df_test = pd.DataFrame({'dep': np.array(['01', 21, '2A', '2B', '196'])})
result = replace_str_id(df_test, 'dep')
solution = [1, 21, 201, 202, 196]
print('computed: {}\nexpected: {}'.format(np.array(result['dep']),
                                          solution))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# I convert 'dep' and 'reg' to int values
df = replace_str_id(df, 'dep')
df = replace_str_id(df, 'reg')

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# I would like to present results per department or per region.  I will
#  build a dictionary to be able to link id of a department to the name
#  of a department. I build a function to be able to get a dictionary
#  either of department or for region. Then, I will add a function
#  specially for department.

def get_area_names(df, aera_id_label:str, aera_name_label: str):
    '''
    DESCRIPTION
        build a dictionary of a type or area (department or region)
         allowing getting a name of this area from its id.
    INPUT
        df is the globale dataframe
        aera_id_label is the id (str) of the area: 'dep' (department)
                      or 'reg' (region);
        area_name_label is the name (str) of the area: 'lib_dep'
                        (department) or 'lib_reg' (region).
    OUTPUT
        dict_areas is a dictionary: id is the key, name is the value.
    '''

    dict_areas = dict()
    # get the list of area id and related names
    area_ids = np.array(df[aera_id_label])
    area_names = np.array(df[aera_name_label])
    area_id_uniq = []
    for i, area_id in enumerate(area_ids):
        if area_id not in area_id_uniq:
            area_id_uniq.append(area_id)
            dict_areas[area_id] = area_names[i]
        else:
            pass

    return dict_areas

# Check the function
# - Get a dictionary of departments
dict_lib_dep = get_area_names(df, 'dep', 'lib_dep')
print('Department dictionary:\n', dict_lib_dep)
# - Get a dictionary of regions
dict_lib_reg = get_area_names(df, 'reg', 'lib_reg')
print('Region dictionary:\n', dict_lib_reg)

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# Then I build a function dedicated to the identification of a name of
#  the department according to its id. 

def get_dep_name(dep_id):
    '''
    DESCRIPTION
        give the name of a department in accordance with its id
    INPUT
        dep_id is the id (int) of the department from
               which we want the name.
    OUTPUT
        dep_name is the name (str) of the department related to the
                 given id.
    '''
    try:
        name = dict_lib_dep[dep_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats

t_int = get_dep_name(45)
t_cor = get_dep_name(201)
t_unk = get_dep_name(600)
print('Loiret: {} - Corse-du-Sud: {} - unknown: {}'.format(t_int, t_cor, t_unk))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# Then I build a function dedicated to the identification of a name of
#  the region according to its id. 

def get_reg_name(reg_id):
    '''
    DESCRIPTION
        give the name of a region in accordance with its id
    INPUT
        reg_id is the id (str, int or float) of the region from which
               we want the name.
    OUTPUT
        reg_name is the name (str) of the region related to the given
                 id.
    '''
    try:
        name = dict_lib_reg[reg_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats
t_int = get_reg_name(93)
t_unk = get_reg_name(604)
solution = "Provence-Alpes-Côte d'Azur"
print("{}: {} - unknown: {}".format(solution, t_int, t_unk))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# Then by extension, I build a function to provide the name of
#  departments from a list of their ids.

def get_dep_names(dep_ids: list):
    '''
    DESCRIPTION
        give a list of department names according to a list of
         department ids.
    INPUT
        dep_ids is the list of department id
    OUTPUT
        dep_names is the list of department names
    '''
    dep_names = list()
    for dep_id in dep_ids:
        dep_names.append(get_dep_name(dep_id))

    return dep_names

# Check the function
list_dep_ids = [45, 21]
solution = ['Loiret', "Côte-d'Or"]
print('expected: {} vs computed: {}'.format(solution,
                                        get_dep_names(list_dep_ids)))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# In the same way, I build a function to provide the name of region from
#  a list of their ids.

def get_reg_names(reg_ids: list):
    '''
    DESCRIPTION
        give a list of region names according to a list of region ids
    INPUT
        reg_ids is the list of region id
    OUTPUT
        reg_names is the list of region names
    '''
    reg_names = list()
    for reg_id in reg_ids:
        reg_names.append(get_reg_name(reg_id))

    return reg_names

#  Check the function
list_reg_ids = [76, 27]
solution = ['Occitanie', 'Bourgogne et Franche-Comté']
print('expected: {} vs computed: {}'.format(solution,
                                        get_reg_names(list_reg_ids)))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Building Department & Region dictionaries

# Finally I can remove labels of department and region since I have
#  their dictionaries.
size_before = df.shape
df.drop(columns=['lib_dep', 'lib_reg'], inplace=True)

# check result of the action:
title = "df's size has changed from"
print("{} {} to {}:".format(title, size_before, df.shape))

# ----------------------------------------------------------------------

DATA CLEANING - Adding timestamp

For visualization, I need to convert string dates into an int value of
 number of days (timestamp). For that purpose I will split the string
 into the year, the month and the day and compute the number of passed
 days for every of these part since their 0-ref.

For this purpose, I need:

- A function that split the string dates and convert this into the
  equivalent number of days passed since days 0 of year 0; for that
  purpose, I need to convert year and month in number of passed days.
  
- A function that gives the number of passed days to beginning of the
  selected month from the beginning of the year; I can use a kind of
  calendar that gives the number of passed from beginning of the day
  for every month.

For modeling, I could use the timestamp but do not use the 'date'
 (str). I shall remove the column 'date' from the dataframe dedicated
 for modeling.

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Adding timestamp

# Create a kind of calendar that provide the number of cumumlated passed
#  days at beginning of every month since beginning of the year.

def get_cumul_days():
    '''
    DESCRIPTION
        Return a list to ease count, month per month, passed day from
         the beginning of the year.
    INPUT
        nil
    OUTPUT
        cnt_day_at_start_month is a list that gives the nb of past days
         from beginning of the year for every month.
    '''
    cnt_day_at_start_month = []

    day_by_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    cnt = 0

    for i in day_by_month:
        cnt += i 
        cnt_day_at_start_month.append(cnt)

    return cnt_day_at_start_month

# Chech the function
calendar = get_cumul_days()
print('calender:', calendar)

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Adding timestamp

# Then I use this calendar to build a function that will provide the
#  number of passed days since beginning of the year to beginning of
#  the selected month.

def get_cnt_day_at_start_month(m, calendar):
    '''
    DESCRIPTION
        get the number of past days from the beginning of year and at 
         beginning of the selected month.
    INPUT
        m is the selected month (str of int, int or float)
        calendar is a list that gives the nb of past days from
                 beginning of the year for every month
    OUTPUT
        cnt is the count of days (int) from beginning of the year at
         start of the selected month.
    '''
    cnt = calendar[int(m)-1]
    return cnt

# Check the function
# 31 days passed at beginning of february.
result = get_cnt_day_at_start_month(2, calendar)
print('result: {} vs expected: {}'.format(result, 31))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Adding timestamp

# Finally, I compute a timestamp from the dates as the number of days
#  passed  from 0 to the selected day such as for instances:
# 2022-06-24 => 2022 * nb_days_passed_to_this_year from year 0
#               + nb_passed_days_from_begin_of_the_year_to_begin_of
#                 this_month
#               + 24 (nb of days passed since beginning of the month)
#               = 2022 * 365.24219 + 151 + 24 = 738695
# 0000-01-01 => 0 + 0 + 1 = 1


def get_timestamp_array(dates_str):
    '''
    DESCRIPTION
        Convert an np.array of dates in string format
        to a timestamp in number of days.
    INPUT
        dates_str is a np.array of dates YYYY-MM-DD in string format
    OUTPUT
        timestamp is np.array with data converted in number of days (int)
    '''
    timestamp = []
    j = 0
    for d in dates_str:
        try:
            y = float(d[:4])
            m = float(d[5:7])
            d = float(d[8:10])
            # convert the data in number of days
            cnt_days_for_months = float(get_cnt_day_at_start_month(m, calendar))
            ts = y * 365.24219 + cnt_days_for_months + d
        except:
            ts = 0.0
        ts_r = round(ts, 0)
        timestamp.append(int(ts_r))
        j += 1
    return np.array(timestamp)

# Check the function
test_dates = np.array(['2022-06-24', '0000-01-01'])
timestamp_arr = get_timestamp_array(test_dates)
print('expected: {} vs computed {}'.format([738695, 1], timestamp_arr))

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - Adding timestamp

# Add a timestamp column in df
# It will hep for displaying data in time series and find most recent
#  values per department.
df['timestamp'] = get_timestamp_array(np.array(df['date']))
# update the labels dictionary
df_dict_labels['timestamp'] = 'Timestamp'
# As a reminder, I've decided to keep the column 'date' for avoid 
#  need to rebuild the date value in string format from the timestamp
#  when needed.

# ----------------------------------------------------------------------

DATA CLEANING - analysis of residual missing values

Analyze content of the dataframe to detect invalid data and decide if
 a can keep them or no.


In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - analysis of residual missing values

display_columns_info(df)

# ----------------------------------------------------------------------

DATA CLEANING - analysis of residual missing values

The function raises that following labels have 0.27% missing values:
tx_pos, tx_incid, pos, pos_7j

I assume we can remove rows with missing values in these columns without
 a significant impact on forcoming results.

In [ ]:
# ----------------------------------------------------------------------
# DATA CLEANING - remove residual missing values

size_before = df.shape
# I remove all rows containing at least one nan
df.dropna(axis='index', how='any', inplace=True)

# check result of the action:
title = "df's size has changed from"
print("{} {} to {}:".format(title, size_before, df.shape))

# Check result of the action
display_columns_info(df)

# ----------------------------------------------------------------------

# DATA ANALYSIS - time

I would compare the time period covered by every data file to illustrate
 their differences.

I need to add timestamp column in every data file ,
 then order the dataframe on this timestamp.

In [ ]:
# ----------------------------------------------------------------------
# DATA ANALYSIS - time

# Add timestamp to every dataframe
# df_file_3, df_file_4, df_file_2, df_file_1

df_file_3['timestamp'] = get_timestamp_array(
                              np.array(df_file_3['date']))
df_file_4['timestamp'] = get_timestamp_array(
                              np.array(df_file_4['date']))
df_file_2['timestamp'] = get_timestamp_array(
                              np.array(df_file_2['date']))
df_file_1['timestamp'] = get_timestamp_array(
                              np.array(df_file_1['date']))

In [ ]:
# ----------------------------------------------------------------------
# DATA ANALYSIS - time

# Build a function to link timestamp to date

def get_df_related_value(df, label_in, label_out, value_in_ref):
    '''
    DESCRIPTION
        return value of a column from a dataframe at the same
         index that I find a given value in another column of 
         this dataframe.
    INPUT
        df is a pandas dataframe;
        label_in (string) is the label of the column where to search
                 the value value_in_ref;
        label_out (string) is the label of the column from where
                  I return the value at the same index where I find
                  value_in_ref;
        value_in_ref (str, int or float) is the value to search for in
                     column named label_in.
    OUTPUT
        value_out (str, int or float) is the value from column
                  label_out at the index where I find the value_in_ref.

    comment:
        I take the first position found. 
    '''
    value_out = np.nan

    for i, value in enumerate (np.array(df[label_in])):
        if value == value_in_ref:
            value_out = np.array(df[label_out].values)[i]
            break
    return value_out

# ----------------------------------------------------------------------

DATA PROCESSING - prepare dataframe for processing and analysis


Before starting data processing, I shall:
- I group the dataframe by department (if no dummy method applied to 
  the departments) and by time (timestamp).
- Create a list of unique departments for preparing the analysis per
  department.
- I've indicated that the dataframe shall be copied to own a dataframe
  dedicated for modeling.

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - Prepare dataframe
# 
# Group the data set by dep and by timestamp to ease searching results by
#  department and by time.

if 'dep' in df.columns:
    df.groupby(['dep', 'timestamp'])  # If no dummy applied
else:
    df.groupby(['timestamp'])  # If dummy applied

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - Prepare plotting

# Get the unique list of departments to get results for every department
deps = pd.unique(df['dep'])

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - Prepare modeling

# As mentioned before, last action of the cleaning phase consists in 
#  copy the dataframe to have a specific dataframe for modeling while 
#  the other dataframe will be used for calculation and addition of
#  data.

# Copy df for modeling
df_mdl = df.copy(deep=True)

# As mentioned before, remove the column 'date' (str):
df_mdl.drop(columns='date', inplace=True)

# ----------------------------------------------------------------------

DATA PROCESSING - Define the needs

According to this description, it may be possible to compute the number
 of people in the population as
 n_cum_dose1 / couv_dose1 = pop  (new)
I choose this mean because n_cum_dose1 must be higher than other
 n_cum_xxx such I get the highest accuracy possible with this data set.

We can calculate:
-	the population, label ‘pop’:
  pop = n_cum_dose1 / couv_dose1
I choose this mean because n_cum_dose1 must be higher than other n_cum_xxx such I get the highest accuracy possible with this data set.
I will check it did not vary too much along the time

-	the rate of the population which is fully vaccinated (two doses), label ‘tx_f_vacc’:
  tx_f_vacc = n_cum_complet / pop

-	the rate of the population which is hospitalized, label ‘tx_hosp’;
  tx_hosp  = hosp / pop

-	the rate of population positive to COVID-19, label ‘tx_pos’:
  tx_pos = pos / pop

-	the rate of people in resuscitation or intensive care, label ‘tx_rea’:
  tx_rea = rea / pop

-	the rate of mortality at hospital, label ‘tx_dchosp’:
  tx_dchosp = dchosp / pop


Since I'm working with a cleaned dataframe, I'd try keeping this
 dataframe as clean as possible.
I will ensure that adding computed values does not bring invalid data
 (nan or infinite). I can ensure that by adding systematically a
 function to clean the result of a computation.
 ACTION: Build a function to clean result of computation.

 I also need to prepare data for visualization
 ...
 ...

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - ensure computed data validity

# This function allows removing unexpected values from new computed
#  columns.

def replace_invalid_data(df, label):
    '''
    DESCRIPTION
        Replace invalid data like nan or infinite by values.
    INPUT
        df is the dataframe to work on;
        label of the category (string) where invalid data will be
         replaced.
    OUTPUT
        df is the dataframe modified.
    '''
    # Replace nan values by 0
    df[label] = df[label].fillna(0)

    # get the maximum ot he values which is non-infinite
    #  to ensure providing a finite value as maximum when needed
    serie = list()
    for val in df[label]:
        if val != np.inf:
            serie.append(val)
    maxi = max(serie)
    mini = min(serie)
    # Replace infinite (inf) values by the min/max of the serie
    df[label].replace({np.inf: maxi, -np.inf: mini}, inplace=True)  

    return df

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Since a use several times the operation of division with other 
# functions. I can build a specific function

def divide_df(df, label_num, label_denum, label_result,
              description=False):
    '''
    DESCRIPTION
        Divide two columns of a dataframe according to their label 
        and push the result into the df undder a new label
    INPUT
        df is the pandas dataframe;
        label_num (string) is the column's label used for numerator 
                  of the division;
        label_denum (string) is the column's lael used for denumerator 
                    of the division;
        label_result (string) is the label of the new column created
                     for implementing result of the division;
        description (bool) is the status to display (True) or not
                    (False) the statistical description of the new 
                    column: Default = False.
    OUTPUT
        df is the input pandas dataframe completed with the new colum
           providing result of the devision.
    comment:
        div function allows supporting missing value (in case I miss
         something) with fill_value.
    '''

    # do the computation into the new column
    df[label_result] = df[label_num].div(df[label_denum])
 
    # Replace invalid data
    df = replace_invalid_data(df, label_result)

    # Display statistics related to the new column
    if description:        
        print("Statistics related to '{}':\n{}".format(label_result,
                                        df[label_result].describe()))

    return df

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the number of people in the population according to
#  pop = n_cum_dose1 / couv_dose1
df = divide_df(df, 'n_cum_dose1', 'couv_dose1', 'pop', description=True)
# Multiply by 100 to compensate couv_dose1 given in %
df['pop'] = np.multiply(100.0, np.array(df['pop']))
# update the labels dictionary
df_dict_labels['pop'] = 'Population'

# Description of the file in the web site indicate that 
#  the reference Population considered is the one given by INSEE
#  at 1st January 2020, at 1st January 2021 and at 1st January 2022

In [ ]:
population_last = np.array(df[df['dep']== 31]['pop'])[-1]
print(population_last)

# ----------------------------------------------------------------------

DATA PROCESSING - Discussion

It appears that some values of 'pop' give zero due to 0-value for at least one
 of both values used for the calculation.

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of people fully vaccinated
#  tx_f_vacc = n_cum_complet / pop
df = divide_df(df, 'n_cum_complet', 'pop', 'tx_f_vacc', description=True)
# Convert the result in %
df['tx_f_vacc'] = np.multiply(100.0, np.array(df['tx_f_vacc']))
# update the labels dictionary
df_dict_labels['tx_f_vacc'] = 'Rate of population fully vaccinated (%)'

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of hospitalization
#  tx_hosp = hosp / pop
df = divide_df(df, 'hosp', 'pop', 'tx_hosp', description=True)
# Convert the result in %
df['tx_hosp'] = np.multiply(100.0, np.array(df['tx_hosp']))
# update the labels dictionary
df_dict_labels['tx_hosp'] = 'Rate of hospitalization (%)'

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of COVID-19 positive 
#  tx_hosp = pos / pop
df = divide_df(df, 'pos', 'pop', 'tx_pos', description=True)
# Convert the result in %
df['tx_pos'] = np.multiply(100.0, np.array(df['tx_pos']))
# update the labels dictionary
df_dict_labels['tx_pos'] = 'Rate of COVID-19 positive (%)'

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of hospitalization
#  tx_hosp = hosp / pop
df = divide_df(df, 'tx_hosp', 'tx_pos', 'rate_conv_hosp', description=True)
# update the labels dictionary
df_dict_labels['rate_conv_hosp'] = 'Rate of conversion from positive to hospitalization'

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of people in resuscitation or intensive care 
#  tx_rea = rea / pop
df = divide_df(df, 'rea', 'pop', 'tx_rea', description=True)
# Convert the result in %
df['tx_rea'] = np.multiply(100.0, np.array(df['tx_rea']))
# update the labels dictionary
title = 'Rate of people in resuscitation or intensive care (%)'
df_dict_labels['tx_rea'] = title

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of people in resuscitation or intensive care 
#  tx_dchosp = dchosp / pop
df = divide_df(df, 'dchosp', 'pop', 'tx_dchosp', description=True)
# Convert the result in %
df['tx_dchosp'] = np.multiply(100.0, np.array(df['tx_dchosp']))
# update the labels dictionary
df_dict_labels['tx_dchosp'] = 'Rate of death at hospital (%)'

# ----------------------------------------------------------------------

DATA PROCESSING - Preparing visualization

A part of the data processing consists in preparating and formating 
 data for visualization.

I shall prepare two types of visualizations:
- time series
- maps (result per departments) 


In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='pop', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='tx_f_vacc', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='tx_hosp', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='tx_pos', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='rate_conv_hosp', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='tx_rea', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - TIME SERIES

# Plot time series
d_list = [9, 12, 31, 32, 46, 65, 81, 82]
plot_time_series(df, label='tx_dchosp', departments=d_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - Preparing visualization for maps

# I want to display computed values at the most recent date
#  so for that purpose, I need to identify the last date and corresponding 
#  values for the selected category (column).

def get_values_at_last_date(df, category, deps):
    '''
    DESCRIPTION
        Provide values of the selected category at the last date available in
         the dataframe, for every department.
    INPUT
        category is the name of the columns for which we want values at the 
         last date for every department
    OUPUT
        values is a dictionary with {dep: {'date': date, 'value': value of the
         category at the stored date}}
    '''

    values = dict()

    # format a dedicated list of data for the search
    lst = [np.array(df['dep']),
        np.array(df['date']),
        np.array(df['timestamp']),
        np.array(df[category])]

    # Run over every department
    for dep in deps:
        time_ts = 0
        id_max_date = 0
        # run along the rows of the dataframe
        for i, j in enumerate(lst[0]):
            # When I met the selected department
            #  and the timestamp is higher (ensure get finally the most
            #  recent timestamp/date and related values of the category),
            #  we stored date and value of the category in values, relatively
            #  to the department. 
            if (j == dep) and (time_ts < lst[2][i]):
                id_max_date = i
                time_ts = lst[2][i]

        values[dep] = {'date': lst[1][i], 'value': lst[3][i]}
    return values

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - Preparing visualization for maps

# Convert result for visualization, I need to get values like
#  date, {dep: values, ...}

def format_values_for_vizu(dico_values, nb_decimal):
    '''
    DESCRIPTION
        format values for a vizualization on a map
    INPUT
        dico_values is a dictionary such {dep: {'date': date, 'value': value
         of the category at the stored date}}
    OUTPUT
        series is a dictionary such as {date1: {depY: valueY, ...}, date2: {...}}
    '''

    series = dict()

    dates = list()
    for dep, j in dico_values.items():
        date = j['date']
        if date not in dates: # Create a sub-dictionary at every new date
            dates.append(date)
            series[date] = dict()
        series[date][dep] = round(j['value'], nb_decimal)

    return series

In [ ]:
# ----------------------------------------------------------------------     CHECK IS STILL USEFUL
# DATA PROCESSING - Preparing visualization for time series

# I need to get all values of a categories along the time for a selected department

def get_timeserie(category, dep, duration_week):
    
    # get dataframe the selecte category with time and departments
    df_ = df[['dep', 'date', 'timestamp', category]]

    # keep only rows with the selected dep
    df_ = df_[df_['dep'] == dep]

    # sort dataframe by descending timestamp (most recent at first)
    df_.sort_values(by=['timestamp'], ascending=False, inplace=True)
    
    # get last timestamp (most recent date)
    timestamp_max = df_['timestamp'].values[0]

    # compute start of the time window
    timestamp_min = timestamp_max - 7*duration_week  # unit here is the day
                                                     # 7 days per week

    # keep only dataframe on the required time stamp period
    df_ = df_[df_['timestamp'] > timestamp_min]

    # get expected values and related dates
    dates = df_['date'].values.tolist()
    values = df_[category].values.tolist()
    # revers is for getting time flow from left to right

    return list(reversed(dates)), list(reversed(values))

In [ ]:
# ----------------------------------------------------------------------
# DATA VISUALIZATION - PLOTTING ON MAP

# Build a generic function for plotting figures on a map per department

def plot_last_figures_on_map(df, label):
    '''
    DESCRIPTION
        Provide the final data set to display figures on a map per
         departement.
    INPUT
        df is the pandas dataframe
        label (string) is the name column's name for which I want to
              display figures.
    OUTPUT
        nil
    '''
    # format for vizualization
    last_values_per_dep = get_values_at_last_date(df, label, deps)
    # Get the selected data
    values_visu = format_values_for_vizu(last_values_per_dep, 3)
    # Data for visualization
    print('{}:\n{}'.format(label, values_visu))

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - DUMMY
# 
# Try to get dummies on dep
# Since there is a lot, wait and see.

def dummy_data(df, cat):
    '''
    DESCRIPTION
        add dummy data to df removing the source of dummy
    INPUT
        df is the initial dataframe to work on
        cat is the name of the category we want dummying
    OUTPUT
        df_dummy is the dataframe as copy of df but with dummy columns,
         removing the source of dummy
    '''
    df_dummy = df.copy(deep=True)

    # Create a new dataframe for preparing dummies from a copy of the 
    #  selected category 
    df_cat = pd.DataFrame([])
    df_cat[cat] = np.array(df[cat])
    print('shape df_cat:', df_cat.shape)

    # Convert category values to just numbers 0 or 1 (dummy values)
    #  of category
    df_dum = pd.get_dummies(df_cat[cat], dtype=int)
    print('shape df_dum:', df_dum.shape)

    for col in df_dum.columns:
        df_dummy[col] = np.array(df_dum[col])
    print('shape df_dummy:', df_dummy.shape)

    # Drop the original category column from `df_dummy`
    df_dummy.drop(columns=cat, inplace=True)
    print('shape df_dummy drop:', df_dummy.shape)

    # check number of duplicates
    #  before trying removing duplicates
    duplicates = df_dummy.duplicated().sum()
    if duplicates > 0:
        df_dummy.drop_duplicates(keep='first', inplace=True)
    print('shape df_dummy dupl:', df_dummy.shape)

    # clean df_dummy from rows with full nan values
    #  otherwise, at least the row is a nana row.
    df_dummy.dropna(axis='index', how='all', inplace=True)
    print('shape df_dummy dropna:', df_dummy.shape)

    return df_dummy


# Add dummy values of 'dep'
# df_mdl = dummy_data(df_mdl, 'dep')
# df.head()


In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - CLEANING

# Clean the dataframe dedicated to model 
print(df_mdl.shape)

# Remove the date (str) and dep (str) not supported for
#  the linear regression model.
cols = ['date', 'dep', 'reg', 'dchosp', 'rad', 'incid_rad']
for col in cols:
    try:
        df_mdl.drop(columns=col, inplace=True)
    except:
        pass

# Remove unexpected values from the dataframe dedicated to model
df_mdl.dropna(axis='index', how='any', inplace=True)

# Check result of the action
print(df_mdl.shape)

In [ ]:
# use timestamp for removing the beginning og the period.
# The analysis of data shows starting data are not reliable.

# I fix the limit to 15-January-2021, i.e. timestamp 738170.

df_mdl = df_mdl[df_mdl['timestamp'] > 738170]
print(df_mdl.shape)

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Build a model

# Split the dataset into explanatory and response data sets

def get_X_y(df: object, response:str):
    '''
    DESCRIPTION
        Split df into exploratory X data and response y data
    INPUT
           df : pandas dataframe
           response : target variable
    OUTPUT
           X : A matrix holding all of the variables you want to
               consider when predicting the response
           y : the corresponding response vector
    '''
    # output
    X, y = None, None

    # Get the Response var
    if response in df.columns:
        # Split into explanatory and response variables (1/2)
        #  Get response variable
        y = df[response]
        df = df.drop(columns=[response])  # Remove pred_name from df
    else:
        print("\tCAUTION! Unable to find the response in df")
        y = None

    # Get the Exploratory vars
    # Split into explanatory and response variables (2/2)
    #  Get the input variables i.e. at this level just a copy of df
    X = df.copy(deep=True)


    if X is None:
        print('\tCAUTION! No X found')
    if y is None:
        print(' - No y found')

    del df, response
    return X, y

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Get a trained model

def get_model(X: object, y: object, testrate=.3):
    '''
    DESCRIPTION
        Returns a linear prediction model according to train data
    INPUT
           X          : explanatory variables object
           y          : response variable object
           testrate   : proportion of the dataset to include in
                         the test split,between 0.0 and 1.0;
                         default value = 0.3
    OUTPUT
            model : linear regression model object from sklearn
            list of X_train and y_train
            list of X_test and y_test
    '''
    model = None
    Xtrain, Xtest, ytrain, ytest = None, None, None, None

    # sub-variables
    y_pred, split = None, False
    
    if (X is not None) and (y is not None):
        # Split into train and test X/y data set
        #  to establish the model and score it
        print("- Training")
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                        test_size=testrate,
                                                        random_state=42)
        if (Xtrain is not None) and (Xtest is not None) and \
           (ytrain is not None) and (ytest is not None):
            split = True

        # Establish model
        print("- Modeling")
        if split:

            # Linear model from scikit-learn:
            #  https://scikit-learn.org/stable/modules/linear_model.html#

            # Linear Regression
            model = LinearRegression()

            # Linear model Lasso
            # model = linear_model.Lasso(alpha=0.1)

            # Ridge regression and classification
            # model = linear_model.Ridge(alpha=.5)

            # BayesianRidge()
            # model = linear_model.BayesianRidge()

            # LogisticRegression
            # model = linear_model.LogisticRegression(random_state=0)
            # very long -> not achieve

            # generalized linear model TweedieRegressor
            # model = linear_model.TweedieRegressor(power=1, alpha=0.5, link='log')
            # Does not work!

            # Stochastic Gradient Descent
            # model = linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
            # Does not provide good result

            # model  = linear_model.Perceptron(tol=1e-3, random_state=0)
            # Does not provide good result

            # fit the model
            model.fit(Xtrain, ytrain)

            # Test the model on test data
            y_pred = model.predict(Xtest)
        else:
            y_pred = None
            print("\tCAUTION! No Model defined")
 
    return model, [Xtrain, ytrain], [Xtest, ytest]

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Evualuate the performance of the model

def get_model_performance(model, Xy_train, Xy_test, Xy):
    '''
    DESCRIPTION
        Evaluate the model by computing its R2 score.
    INPUT
        model is the model to evaluate
        Xy_train is a list of train data: [X_train, y_train]
        Xy_test is a list of test data: [X_test, y_test]
        Xy is a list of with all data: [X, y]
    OUTPUT
        score (float) in the measure of the model's performance
    '''

    # Get data sets
    X_train, y_train = Xy_train
    X_test, y_test = Xy_test
    X, y = Xy

    # Evaluate this model by gettings metrics with a model by
    #  regression
    print("- Metrics:")
    
    # Get the model's score
    # Return the coefficient of determination of the prediction
    # mdl_score_train = model.score(X_train, y_train)
    # print("  > Model score (train):", mdl_score_train)
    # mdl_score_test = model.score(X_test, y_test)
    # print("  > Model score (test):", mdl_score_test)
    # mdl_score = model.score(X, y)
    # print("  > Model score (all) :", mdl_score)

    # Accuracy_score (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.accuracy_score.html)
    # Confusion matrix (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.confusion_matrix.html)
    #  not appropriate for my purpose
    # Common pitfalls (https://scikit-learn.org/stable/common_
    #  pitfalls.html): mean_sqaured_error and r2_score

    # According to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # ... Accuracy score is only for classification problems
    # acc_score_train = accuracy_score(y_train, model.predict(X_train))
    # acc_score_test = accuracy_score(y_test, model.predict(X_test))
    # if debug: print("    - acc_score_train:", acc_score_train)
    # if debug: print("    - acc_score_test:", acc_score_test)

    # always according to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # For regression problems, use: R2 Score, MSE (Mean Squared Error),
    # RMSE (Root Mean Squared Error).
    
    # r2_score_train = r2_score(y_train, model.predict(X_train))
    # r2_score_test = r2_score(y_test, model.predict(X_test))
    r2_score_all = r2_score(y, model.predict(X))
    # print("  > r2 score (train):", r2_score_train)
    # print("  > r2 score (test):", r2_score_test)
    # print("  > r2 score (all) :", r2_score_all)

    score = r2_score_all
    print("  > Model's score: ", score)
    
    return score

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING -  Indicate the weight of every parameters
#  in this model

def coef_weights(model, X) -> object:
    '''
    DESCRIPTION
       Returns a dataframe with coefficients of the model
        (real and absolute values) sorted in the descending order
        of the absolute values
    INPUT       
       model is the model for which we are looking coefficients
       X is the exploratory set of data
    OUTPUT
      coefs_df : model's coefficients that can be used to understand
                 the most influential coefficients in a linear model
                 by providing the coefficient estimates along with the
                 name of the variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    # Get name of every column in front  of its coefficients
    coefs_df['est_int'] = X.columns
    # get coefficients of the linear model
    coefs_df['coefs'] = model.coef_
    # get absolute value of these coefficients
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    # Sort coefficient by descending order
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)

    return coefs_df

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Define, evualute and discuss the model

def process_model(df, response, drop_list=[]):
    '''
    DESCRIPTION
        Define, evaluate and discuss a model
    INPUT
        df is the pandas dataframe with data to build and evaluate the 
           model;
        response (string) is the category / label of the column defined
                 as response/target for the model to define;
        drop_list (list) is a list of categories / labels of the columns
                  that can be removed from the dataframe before 
                  defining and evaluating the model. It does not modify
                  input dataframe. Default = nil (empty list)
    OUTPUT
        model is the trained model define to predict the response
              according to the other labels of the dataframe.
        score is the metrics to measure the performance of the model.coef_
              closer to 1, better.
        coefs is the list of labels of the dataframe used to establish
              the model for which a weight (contribution to the model) is
              computed (signed and absolute values).
    '''
    df_mdl = df.copy(deep=True)
    print('Modeling...')
    # Remove this list of labels
    print('- drop')
    if len(drop_list) > 0:
        df_mdl.drop(columns=drop_list, inplace=True)

    # Split into Response y / Exploratory variables X
    print("- Split into X/y")
    X, y = get_X_y(df_mdl, response)

    # modeling
    print("- Get model")
    model, Xy_train, Xy_test = get_model(X, y)

    # Check performance
    print("- Performance")
    score = get_model_performance(model, Xy_train, Xy_test, [X, y])

    # Coefficients of weight
    print("- Model's coefficients")
    coefs = coef_weights(model, X)
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print(coefs)

    return model, score, coefs

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Apply the model

# define the target
response = 'hosp'
drop_list = []
# Process the target
model, score, coefs = process_model(df_mdl, response, drop_list)

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - refine the dataframe

# define the target
response = 'hosp'

# According to the analysis of the weights of every labels,
#  I propose to remove some of them with neglictible weight
# < 0.1 in absolute value
drop_list2 = ['couv_rappel', 'couv_2_rappel',
                'n_dose1', 'n_rappel',
                'n_cum_dose1',
                'n_complet', 'n_cum_complet',
                'n_2_rappel', 'n_3_rappel',
                'n_cum_rappel', 'n_cum_2_rappel', 'n_cum_3_rappel',
                'n_rappel_biv', 'n_cum_rappel_biv'
               ]

# Process the target
model2, score2, coefs2 = process_model(df_mdl, response, drop_list2)

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Apply the model

# define the target
response = 'TO'
drop_list = []
# Process the target
model3, score3, coefs3 = process_model(df_mdl, response, drop_list)

In [ ]:
# add the rate of hospitalization

# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the number of people in the population according to
#  pop = n_cum_dose1 / couv_dose1
df_mdl = divide_df(df_mdl, 'n_cum_dose1', 'couv_dose1', 'pop', description=True)
# update the labels dictionary
df_dict_labels['pop'] = 'Population'

# ----------------------------------------------------------------------
# DATA PROCESSING

# Compute the rate of hospitalized epople
#  tx_hosp = hosp / pop
df_mdl = divide_df(df_mdl, 'hosp', 'pop', 'tx_hosp', description=True)
# update the labels dictionary
df_dict_labels['tx_hosp'] = 'Rate of hospitalization'

In [ ]:
# ----------------------------------------------------------------------
# DATA PROCESSING - MODELING - Apply the model

# define the target
response = 'tx_hosp'
drop_list = []
# Process the target
model4, score4, coefs4 = process_model(df_mdl, response, drop_list)

Explain the results - Question 5:

   The model to predict COVID-19 positive cases per week gives rather good
   result
    with a global score of 0.88.
   Analysis of the contribution of every parameters shows that the result is
    mainly linked with the Occupancy rate (TO) which is actuall rather 
	consequence of the prediction.
   It shows a high level a correlation between the number of positive cases
    and the amount of work in the hospitals despite the vaccination campaign.

FINDINGS

About 6% of data were removed from the initial data set due to missing 
 values. It concerns mainly first set of data at the beginning of the data 
 collection. Indeed, method, measure, collection and presentation of data 
 change in time before getting the current format of data.
 Nevertheless, we benefit from a huge amount of valid data for the analysis.
 
Computation of some rates raised the problem of division by 0. While taking 
 about group of people and with the will of showing result for every 
 department, I replace 0 by 1 when necessary.
 
I also face infinite values; then I replaced with infinite values by maximum
 numerical values of the serie, in order to be reasonable realistic and show
 result.

Although testing several type of models, I kept the Linear Regression which
 give rather good results with an implementation rather simple. 
I wonder if some other type of models could be more suitable in this case.

Display of maps and plots from the Notebook was not easy due to difficulties
of installation of additional libraries and other things required to make it 
run correctly; all these installations were not possible on my profressional
laptop to safety policy of my company.
Nevertheless, display of these graph is possible with the python code.